# Projeto 2 - Ciência dos Dados

Nome: Gabriela Choichit Giosa

Nome: João Pedro Faria de Araujo

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [8]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [1]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import pandas as pd
import numpy as np
import re
from IPython.display import display
import string

In [401]:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """ 
    punctuation = '[!\-.:?;,\'@:]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

+___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [402]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

FileNotFoundError: [Errno 2] No such file or directory: 'auth.pass'

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [403]:
#Produto escolhido:
produto = '#Joker'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'en'

Capturando os dados do twitter:

In [404]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

NameError: name 'auth' is not defined

Salvando os dados em uma planilha Excel:

In [405]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

NameError: name 'msgs' is not defined

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

lendo o arquivo e fazendo a tabela de relativos e não relativos

In [406]:
joker = pd.read_excel('#JokerFull.xlsx')
teste = pd.read_excel('#JokerFull.xlsx', 'Teste')
joker_idx = joker.set_index('Treinamento')
joker_relevante = joker_idx[joker_idx.Classe == 1]
joker_nao_relevante = joker_idx[joker_idx.Classe == 0]
teste_idx = teste.set_index('Teste')
teste_idx_2 = set(teste_idx.index)

filtrando o teste

In [407]:
teste_palavras = str(teste_idx_2)
teste_palavras = cleanup(teste_palavras)
teste_palavras = teste_palavras.split()

joker total

In [408]:
lista_palavras = str(set(joker_idx.index))
lista_palavras =cleanup(lista_palavras)
lista_palavras = lista_palavras.split()
lista_palavras

['{',
 'stealthy',
 'android',
 'trojan',
 'spy',
 'signs',
 'you',
 'up',
 'for',
 'premium',
 'subscriptions\\nvia',
 'bleepincomputer\\n#proficio',
 '#threatnews',
 '#cybersecurity',
 '#mssp',
 '#mdr',
 '#joker',
 '#trojan\\nhttps',
 '//t',
 'co/muyi7huwgn',
 'rt',
 'mysteriostephen',
 'the',
 'dark',
 'knight',
 'promotional',
 '#photoshoot',
 '"',
 '\\n#heathledger',
 '#joker',
 'a',
 'true',
 'legend',
 '"',
 '🃏📸\\n',
 'spider_symbiote',
 'https',
 '//t',
 'co/0kkd83…',
 'the',
 'untitled_cast',
 '#tiff19\\xa0',
 'coverage',
 'continues',
 'with',
 'our',
 'review',
 'of',
 '#joker',
 '\\n\\n40',
 'minutes',
 'of',
 'spoiler',
 'free',
 'impressions',
 '…',
 'https',
 '//t',
 'co/f6fsxddc9z',
 '"rt',
 'comicbook',
 '#joker',
 'director',
 'explains',
 'why',
 'it',
 's',
 'not',
 'a',
 'typical',
 'origin',
 'movie\\nhttps',
 '//t',
 'co/qqh3phmwzh',
 'https',
 '//t',
 'co/sfedovmgvx"',
 '"chinese',
 'citizens',
 'find',
 'trump',
 'perplexing',
 'on',
 'trade',
 'war',
 'china',

In [409]:
serie_total = pd.Series(lista_palavras)
tabela_total_absoluta = serie_total.value_counts()
tabela_total_relativa = serie_total.value_counts(True)
tabela_total_relativa

#joker                    0.036168
//t                       0.034740
https                     0.030933
the                       0.029188
rt                        0.021256
a                         0.017608
to                        0.014435
of                        0.011580
is                        0.010945
i                         0.009042
and                       0.008249
for                       0.007773
this                      0.007297
joker                     0.006980
in                        0.006821
it                        0.006345
you                       0.006345
on                        0.005869
movie                     0.005393
#jokermovie               0.005235
s                         0.005076
"rt                       0.004918
with                      0.003966
joaquin                   0.003966
as                        0.003807
one                       0.003807
be                        0.003807
that                      0.003807
so                  

In [410]:
lista_palavras_relevantes = str(set(joker_relevante.index))
lista_palavras_relevantes = cleanup(lista_palavras_relevantes)
lista_palavras_relevantes = lista_palavras_relevantes.split()
lista_palavras_relevantes

['{',
 'rt',
 'a__g__t',
 'instant',
 'classic',
 '#joker',
 '#jock',
 'https',
 '//t',
 'co/zysqqsq0dk',
 '#joker',
 'seems',
 'like',
 'the',
 'go',
 'to',
 'movie',
 'for',
 'all',
 'future',
 'mass',
 'shooters',
 'comicbook',
 'injustice',
 'fall',
 '#joker',
 'hit',
 'the',
 'feed',
 'for',
 'the',
 'latest',
 'tnr',
 'episode',
 'where',
 'we',
 'are',
 'talking',
 '#joker',
 '#birdsofprey',
 '#themandalorian',
 '#starwars',
 'triple',
 'force',
 'friday',
 'and',
 'much',
 'more',
 '#wethenerd',
 '#staynerd',
 '#weareallclowns',
 '\\n\\n🎧',
 'on',
 'podbeancom',
 'applepodcasts',
 'spotify',
 'or',
 'hit',
 'the',
 'link',
 'below',
 '\\n\\nhttps',
 '//t',
 'co/vnhd1b5i0b',
 'https',
 '//t',
 'co/lxznxttqop',
 'rt',
 'cbr',
 '#jokermovie',
 'makes',
 'a',
 'questionable',
 'soundtrack',
 'selection',
 'https',
 '//t',
 'co/zxpftzdd1y',
 '#joker',
 'https',
 '//t',
 'co/tlzjugaeps',
 '#joker',
 '❤️',
 'waiting',
 '😁😁😁😁',
 'https',
 '//t',
 'co/3mn17gfhc8',
 'rt',
 'ashleygmenzel

In [411]:
lista_palavras_relevantes2 = str(set(joker_relevante.index))

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
lista_palavras_relevantes2 = lista_palavras_relevantes2.translate(translator).split()
lista_palavras_relevantes2

#for k in range(len(lista_palavras_relevantes)):
      #if lista_palavras_relevantes[k] not in lista_palavras_relevantes2:
        #print(lista_palavras_relevantes[k])
lista_palavras_relevantes2

['rt',
 'a',
 'g',
 't',
 'instant',
 'classic',
 'joker',
 'jock',
 'https',
 't',
 'co',
 'zysqqsq0dk',
 'joker',
 'seems',
 'like',
 'the',
 'go',
 'to',
 'movie',
 'for',
 'all',
 'future',
 'mass',
 'shooters',
 'comicbook',
 'injustice',
 'fall',
 'joker',
 'hit',
 'the',
 'feed',
 'for',
 'the',
 'latest',
 'tnr',
 'episode',
 'where',
 'we',
 'are',
 'talking',
 'joker',
 'birdsofprey',
 'themandalorian',
 'starwars',
 'triple',
 'force',
 'friday',
 'and',
 'much',
 'more',
 'wethenerd',
 'staynerd',
 'weareallclowns',
 'n',
 'n🎧',
 'on',
 'podbeancom',
 'applepodcasts',
 'spotify',
 'or',
 'hit',
 'the',
 'link',
 'below',
 'n',
 'nhttps',
 't',
 'co',
 'vnhd1b5i0b',
 'https',
 't',
 'co',
 'lxznxttqop',
 'rt',
 'cbr',
 'jokermovie',
 'makes',
 'a',
 'questionable',
 'soundtrack',
 'selection',
 'https',
 't',
 'co',
 'zxpftzdd1y',
 'joker',
 'https',
 't',
 'co',
 'tlzjugaeps',
 'joker',
 '❤️',
 'waiting',
 '😁😁😁😁',
 'https',
 't',
 'co',
 '3mn17gfhc8',
 'rt',
 'ashleygmenzel

In [412]:

#lista_1 = list(set(joker_relevante.index))
#z = len(list(set(joker_relevante.index)))
#lista_palavras_relevantes2 = []

#for i in range(z):
#    lista_palavras_relevantes2 = lista_palavras_relevantes2 + lista_1[i].split()
#print(len(lista_palavras_relevantes2))
#lista_palavras_relevantes2 = [str(i) for i in lista_palavras_relevantes2]
#lista_palavras_relevantes2 = [''.join(c for c in s if c not in string.punctuation) for s in lista_palavras_relevantes2]
#lista_palavras_relevantes2 = [s for s in lista_palavras_relevantes2 if s]
 


#for k in range(len(lista_palavras_relevantes)):
#      if lista_palavras_relevantes[k] not in lista_palavras_relevantes2:
#        print(lista_palavras_relevantes[k])

Frequencias relevante relativas e absolutas

In [413]:
serie_relevante = pd.Series(lista_palavras_relevantes)
tabela_relevante_absoluta = serie_relevante.value_counts()
tabela_relevante_relativa = serie_relevante.value_counts(True)
tabela_relevante_absoluta

#joker                     153
//t                        134
https                      119
the                        118
rt                          92
a                           76
to                          54
is                          49
of                          44
i                           42
and                         32
movie                       32
this                        31
it                          30
joker                       29
in                          27
for                         27
#jokermovie                 26
as                          22
joaquin                     22
s                           21
be                          18
that                        18
at                          18
"rt                         18
with                        17
you                         17
#joaquinphoenix             17
phoenix                     16
one                         15
                          ... 
osamabinladaan               1
#coringa

lista com as palavras não relevantes filtradas

In [414]:
lista_palavras_nao_relevantes = str(set(joker_nao_relevante.index))
lista_palavras_nao_relevantes =cleanup(lista_palavras_nao_relevantes)
lista_palavras_nao_relevantes = lista_palavras_nao_relevantes.split()
lista_palavras_nao_relevantes

['{',
 'rt',
 'mysteriostephen',
 'the',
 'dark',
 'knight',
 'promotional',
 '#photoshoot',
 '"',
 '\\n#heathledger',
 '#joker',
 'a',
 'true',
 'legend',
 '"',
 '🃏📸\\n',
 'spider_symbiote',
 'https',
 '//t',
 'co/0kkd83…',
 'stealthy',
 'android',
 'trojan',
 'spy',
 'signs',
 'you',
 'up',
 'for',
 'premium',
 'subscriptions\\nvia',
 'bleepincomputer\\n#proficio',
 '#threatnews',
 '#cybersecurity',
 '#mssp',
 '#mdr',
 '#joker',
 '#trojan\\nhttps',
 '//t',
 'co/muyi7huwgn',
 'rt',
 'yousmmashogun',
 'gamebredfighter',
 'give',
 'them',
 '#joker',
 '#gamebred',
 '🔥🔥',
 'https',
 '//t',
 'co/iaizpksd8z',
 'rt',
 'afis8',
 'gonna',
 'have',
 'to',
 'sit',
 'with',
 '#joker',
 'for',
 'a',
 'little',
 'while',
 'but',
 'i',
 'will',
 'say',
 'this',
 'a',
 'lot',
 'of',
 'incels',
 'are',
 'going',
 'to',
 'feel',
 'pretty',
 'fucking',
 'empower…',
 '"slow',
 'burn"\\ntoddphillips1\\n\\none',
 'month',
 'till\\xa0#joker',
 'uk',
 'poster',
 'auckland',
 'cbd',
 'https',
 '//t',
 'co/pqt

Frequencias nao relevantes relativas e absolutas

In [415]:
serie_nao_relevante = pd.Series(lista_palavras_nao_relevantes)
tabela_nao_relevante_absoluta = serie_nao_relevante.value_counts()
tabela_nao_relevante_relativa = serie_nao_relevante.value_counts(True)
tabela_nao_relevante_absoluta

//t                         89
#joker                      84
https                       80
the                         74
rt                          54
to                          40
a                           40
of                          34
is                          23
on                          23
you                         23
for                         22
and                         22
i                           18
in                          18
joker                       17
this                        17
"rt                         15
s                           13
it                          12
one                         11
#batman                     11
\n                          10
by                          10
more                        10
#dc                         10
new                          9
with                         9
from                         9
#dccomics                    9
                            ..
#8waystheworldends           1
leaked  

In [416]:
len(tabela_total_absoluta)

2505

Frequencias nao relevantes e relevantes relativas e absolutas

Priors 
### P(n) e P(r)

In [417]:
Pn = tabela_nao_relevante_absoluta.sum()/tabela_total_absoluta.sum()
Pr = tabela_relevante_absoluta.sum()/tabela_total_absoluta.sum()
Pr, Pn

(0.6080266497461929, 0.434010152284264)

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [418]:
len(joker_relevante.index[0].split())

12

In [419]:
len(joker_relevante.index)


262

In [420]:
t = []
P_tweet_dado_relevante = []


for m in range(len(teste_idx.index)):
    P_tweet_dado_relevante.append(np.prod(t))
    t = []
    for k in teste_idx.index[m].split()[:]:
        if k in tabela_relevante_absoluta:
            t.append((tabela_relevante_absoluta[k] + 1)/(tabela_relevante_absoluta.sum() + len(tabela_total_absoluta.index)))
        else:
            t.append((0 + 1)/(tabela_relevante_absoluta.sum() + len(tabela_total_absoluta.index)))

P_tweet_dado_relevante.pop(+0)
P_tweet_dado_relevante

[9.83332204174516e-61,
 1.5773924421435484e-128,
 1.0866947850517833e-41,
 2.0196219842969322e-18,
 2.0852194712301605e-148,
 3.413889870364528e-64,
 2.713500006942446e-133,
 1.4723069843153493e-33,
 1.1695218591127306e-79,
 2.1462928358911476e-51,
 1.5690062211668533e-45,
 1.9864904764570078e-32,
 6.378041843333303e-80,
 2.3269347843653436e-58,
 2.1472961968424653e-89,
 7.802000670542412e-104,
 1.500979797500318e-67,
 1.2558889497739987e-71,
 2.3269347843653436e-58,
 1.938589048268645e-91,
 4.0485850808379706e-29,
 3.2333583974736714e-27,
 1.563924503226157e-32,
 2.36458865288449e-50,
 9.889472833552714e-59,
 1.2058503765238682e-63,
 6.551157292146041e-139,
 8.694566562776506e-76,
 4.811037458130164e-144,
 5.957039328097626e-69,
 9.827613784937328e-28,
 7.214459032361335e-66,
 9.034140763317327e-52,
 5.528994979141061e-64,
 4.889133895071904e-46,
 1.454473626922183e-208,
 5.629140394553882e-103,
 7.964938745461313e-67,
 1.185598342242101e-79,
 1.8552054800009664e-57,
 6.19339387446788

In [421]:
f = []
P_tweet_dado_nao_relevante = []


for m in range(len(teste_idx.index)):
    P_tweet_dado_nao_relevante.append(np.prod(f))
    f = []
    for k in teste_idx.index[m].split()[:]:
        if k in tabela_nao_relevante_absoluta:
            f.append((tabela_nao_relevante_absoluta[k] + 1)/(tabela_nao_relevante_absoluta.sum() + len(tabela_total_absoluta.index)))
        else:
            f.append((0 + 1)/(tabela_nao_relevante_absoluta.sum() + len(tabela_total_absoluta.index)))

P_tweet_dado_nao_relevante.pop(0)
P_tweet_dado_nao_relevante

[4.220741130636481e-62,
 1.7089567794421776e-131,
 5.848839943825892e-45,
 2.024884081717515e-19,
 7.678751982900989e-147,
 8.221952369751849e-66,
 9.128745653038286e-137,
 2.6092672666543913e-34,
 1.4981931181134564e-80,
 2.2695460423033357e-52,
 1.2841091507352277e-48,
 4.0970499546887955e-34,
 3.542110974367376e-82,
 1.5108782762258378e-57,
 1.2679912735062404e-91,
 2.1441916801789014e-102,
 2.5059170124654305e-66,
 1.7880418588776806e-75,
 1.5108782762258378e-57,
 3.5008888603185173e-93,
 1.7996116338115332e-29,
 4.4854556762076975e-28,
 3.702934312676283e-33,
 1.1084862884670603e-52,
 6.99480683437888e-58,
 1.9302111423886166e-64,
 8.187569660847309e-136,
 2.967820286182988e-80,
 5.528739308788953e-150,
 4.542711610290861e-69,
 1.9241715299534477e-30,
 4.319851247840851e-67,
 1.1148647225349721e-51,
 2.5057774968075925e-62,
 1.9668647351507386e-44,
 6.441995940885916e-206,
 8.565494287966603e-102,
 5.079462242851162e-63,
 1.4715138199319742e-79,
 2.994914431663081e-59,
 5.33982405

In [422]:
i = 0
j = 0
testando = []
teste_2 = []

for k in range(len(P_tweet_dado_relevante)):
    if P_tweet_dado_relevante[k] > P_tweet_dado_nao_relevante[k]:
        i = i + 1
        testando.append(1)
        if testando[k] == teste_idx.Classe[k]:
            teste_2.append('VP')
        else:
            teste_2.append('FN')
    if P_tweet_dado_relevante[k] == P_tweet_dado_nao_relevante[k]:
        print('q')
    elif P_tweet_dado_relevante[k] < P_tweet_dado_nao_relevante[k]:
        j = j + 1
        testando.append(0)
        if testando[k] == teste_idx.Classe[k]:
            teste_2.append('VN')
        else:
            teste_2.append('FP')
    
print(i, j)
#teste_2.value_counts()
print('Verdadeiro Positivo:',teste_2.count('VP'),'\nFalso Negativo:', teste_2.count('FN'),'\nVerdadeiro Negativo:', teste_2.count('VN'),'\nFalso Positivo:', teste_2.count('FP'))
for k in testando:
    print(k)

181 85
Verdadeiro Positivo: 153 
Falso Negativo: 28 
Verdadeiro Negativo: 51 
Falso Positivo: 34
1
1
1
1
0
1
1
1
1
1
1
1
1
0
1
0
0
1
0
1
1
1
1
1
0
1
0
1
1
1
1
1
0
0
0
0
0
0
0
1
0
1
0
1
1
1
1
1
1
0
0
1
0
0
0
0
1
0
1
1
1
1
0
1
1
0
1
0
1
1
1
1
0
0
0
1
0
0
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
1
1
1
1
0
1
1
0
0
0
1
0
1
1
1
0
1
1
1
1
1
1
0
0
1
0
0
0
0
1
0
1
0
1
1
0
1
1
1
0
0
0
1
1
1
1
1
1
0
0
1
1
1
1
0
1
0
0
0
1
0
1
1
0
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
1
1
1
0
0
0
1
1
0
1
1
0
1
0
0
0
0
0
0
1
0
1
0
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
1
1
1
1
0
1
1
0
1


In [423]:
taxa_acerto = (teste_2.count('VP') + teste_2.count('VN'))/(teste_2.count('VP') + teste_2.count('FP') + teste_2.count('VN') + teste_2.count('FN'))

print('taxa de acerto:', round(taxa_acerto, 2)*100)

taxa de acerto: 77.0


___
### Concluindo

##### 1)Faça um comparativo qualitativo sobre os percentuais obtidos para que possa discutir a performance do seu classificador.
    A performance do nosso classificador pode ser avaliada como boa, uma vez que as taxas de Verdadeiros, tanto positivos quanto negativos, foi superior as taxas de Falsos, com 77% de compatibilidade com a classificação feita manualmente.
    Porém podia ser melhorada atraves de diversos processos como a remoção de stopwords, que pode ser feita com o próprio comando do pandas "stopwords", depois de ter limpado os tweets e transformado-os em um texto.
    Além disso tem a função que agrupa palavras semelhantes ou com o mesmo significado(Lemmatizing words), para que só sejam contadas como uma palavra, assim o resultado fica mais próximo do real vez qua algumas que poderiam ser classificadas como novas palavras, enviesando o resultado, entram nesse grupo. 
##### 2)Explique como são tratadas as mensagens com dupla negação e sarcasmo. 
    As mensagens irônicas são tratadas como se não fossem irônias, o classificador não leva em conta o contexto, ele analisa as palavras separadamente, por conta disso também ele considera que uma dupla negativa tem duas vezxes mais chance de ser de uma categoria. 

##### 3)Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto? 
    Eles devem continuar financiando o nosso projeto pois o filme que estamos analisando ainda não foi lançado, então seria interesante fazer uma comparação de tweets antes e depois da estreia, para comparar a espectativa com a opinião real depois de terem assistido o filme, isso para uma produtora seria uma informação muito valiosa.


##### Opcionalmente:  
#####  - Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
    As frases do treinamento são o que dão a base para o classificador, ou seja, as palavras usadas para fazer as tabelas relativas são derivadas dos tweets classificados da base, desse modo, se classificássemos os tweets já no treinamento, os falsos negativos e falsos positivos entrariam para a base de dados como corretos e colaborariam para a propagação do erro.
##### - Propor diferentes cenários de uso para o classificador Naive-Bayes. (Pense em outros cenários sem intersecção com este projeto.)  

    Um fenômeno que também pode ser analisado através do Naive-Bayes é a probabilidade de algum time ser campeão baseado na escalação dos jogadores. Ou seja, baseado em dados coletados de jogos antigos, analisar a performance do jogador e fazer uma previsão de como atuará em campo. 
    Nesse caso porém, como estamos lidando com comportamento de um ser humano, há mais diversos fatores que podem vir a afetar os resultados.

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**